In [138]:
!nvidia-smi

Sun Feb 25 18:18:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              41W / 300W |  12674MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [139]:
%%capture
%pip install datasets
%pip install --upgrade pyarrow

In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
train_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/train.json'
test_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/test.json'


In [142]:
from tqdm import tqdm
from datasets import load_dataset

In [143]:
# chack datasets was installed correctly:
# output: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
# note : the output will vary depending on the time the code is run and the updates in the dataset 'squad'
!python3 -c "from datasets import load_dataset; print(load_dataset('squad', split='train')[0])"

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [267]:
train_data = load_dataset('json', data_files=train_data_path, split='train')
train_data

Dataset({
    features: ['tokens', 'trailing_whitespace', 'full_text', 'labels', 'document'],
    num_rows: 6807
})

In [268]:
train_data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'trailing_whitespace': Sequence(feature=Value(dtype='bool', id=None), length=-1, id=None),
 'full_text': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'document': Value(dtype='int64', id=None)}

In [146]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,O,O,O,O,O,O,O,O,O,B-NAME_STUDENT,I-NAME_STUDENT,O,O,O,O,O,O,O,O,O


In [147]:
import time

start_time = time.time()
text = train_data[0]["tokens"]
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.4f} seconds")

start_time = time.time()
text = train_data["tokens"][0]
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.4f} seconds")

Elapsed time: 0.0018 seconds
Elapsed time: 4.4458 seconds


In [148]:
# remove irrelevant features
train_data = train_data.remove_columns(["full_text", "trailing_whitespace", "document"])
train_data

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 6807
})

In [149]:
len(train_data[0]['tokens']), len(train_data[0]['labels'])

(753, 753)

In [150]:
tags = train_data.features['labels'].feature
tags

Value(dtype='string', id=None)

In [151]:
import datasets

train_data = train_data.cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM'])))
train_data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None)}

In [152]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0


In [153]:
tags = train_data.features['labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None)

In [154]:
# We only need the names for longformer finetunining -> rename all other labels as Other?
# all labels that are not names become 'O'
# we only need the tags 0 , 1, 2

def replace_non_names(example):
  example['labels']
  names = [1, 2]
  name_labels_lists = []
  for label in example['labels']:
    if label in names:
        name_labels_lists.append(label)
    else:
        name_labels_lists.append(0)
  example['name_labels'] = name_labels_lists
  return example

In [155]:
train_data = train_data.map(replace_non_names)

In [156]:
train_data

Dataset({
    features: ['tokens', 'labels', 'name_labels'],
    num_rows: 6807
})

In [157]:
len(train_data[0]['tokens']), len(train_data[0]['labels']), len(train_data[0]['name_labels'])

(753, 753, 753)

In [158]:
train_data.features['name_labels'].feature

Value(dtype='int64', id=None)

In [159]:
train_data = train_data.cast_column("name_labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT'])))
train_data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [160]:
tags = train_data.features['name_labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None)

In [161]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["name_labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0


In [162]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [164]:
tags.num_classes

3

In [165]:
dataset = train_data.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'name_labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'labels', 'name_labels'],
        num_rows: 1362
    })
})

In [166]:
dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [167]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['labels']), len(dataset['train'][0]['name_labels'])

(774, 774, 774)

In [168]:
def remove_col(dataset, col):
  for split in dataset:
    dataset[split] = dataset[split].remove_columns([col])
  return dataset

remove_col(dataset, 'labels')


DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels'],
        num_rows: 1362
    })
})

In [169]:
dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [170]:
# https://huggingface.co/hewonty/longformer-ner-finetuned-pii - fine tuned already
# https://huggingface.co/allenai/longformer-base-4096 - original model
from transformers import AutoTokenizer

lf_model = 'allenai/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', max_length=1024, add_prefix_space=True)


In [171]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels'])

(774, 774)

In [172]:
text = dataset['train'][0]['tokens']
tokenized_input = tokenizer(text, is_split_into_words=True)
lf_tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame({
    'tokens': dataset['train'][0]['tokens'][:10],
    'lf_tokens': lf_tokens[:10],
    'name_labels': dataset['train'][0]['name_labels'][:10]
}).T

,0,1,2,3,4,5,6,7,8,9
tokens,Michael,Ramirez,(,INDIA,),\n\n,VISUALIZATION,\n\n,Elements,:
lf_tokens,<s>,ĠMichael,ĠRamirez,Ġ(,ĠIND,IA,Ġ),Ġ,ĊĊ,ĠVIS
name_labels,1,2,0,0,0,0,0,0,0,0


In [173]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels']), len(lf_tokens)

(774, 774, 824)

In [174]:
len(dataset['train'][0]['name_labels']), len(text), len(lf_tokens)

(774, 774, 824)

In [175]:
# the tokenized 'tokens' result in a longer list - the logformer tokenizer splits into subwords
# we need to align the tokens when we tokenize

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], padding = 'max_length', truncation=True, max_length = 1024, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"name_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [176]:
for split in dataset:
  dataset[split] = dataset[split].map(tokenize_and_align_labels, batched=True)
dataset

Map:   0%|          | 0/1362 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1362
    })
})

In [177]:
pd.DataFrame({
    'tokens': dataset['train'][0]['tokens'][:15],
    'name_labels': dataset['train'][0]['name_labels'][:15],
    'input_ids/tokens': tokenizer.convert_ids_to_tokens(dataset['train'][0]['input_ids'][:15]),
    'labels': dataset['train'][0]['labels'][:15],
}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,Michael,Ramirez,(,INDIA,),\n\n,VISUALIZATION,\n\n,Elements,:,\n\n,1,.,Challenge,:
name_labels,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
input_ids/tokens,<s>,ĠMichael,ĠRamirez,Ġ(,ĠIND,IA,Ġ),Ġ,ĊĊ,ĠVIS,UAL,IZ,ATION,Ġ,ĊĊ
labels,-100,1,2,0,0,-100,0,0,-100,0,-100,-100,-100,0,-100


In [178]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels'])

(774, 774)

In [179]:
len(dataset['train'][0]['input_ids']), len(dataset['train'][0]['labels'])

(1024, 1024)

In [180]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1362
    })
})

In [181]:
dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [182]:
remove_col(dataset, 'tokens')
remove_col(dataset, 'name_labels')
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1362
    })
})

In [183]:
dataset['train'].features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [233]:
for split in dataset:
  dataset[split] = dataset[split].cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
  'B-NAME_STUDENT',
  'I-NAME_STUDENT'])))
dataset['train'].features

Casting the dataset:   0%|          | 0/5445 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1362 [00:00<?, ? examples/s]

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [238]:
pd.DataFrame({
    'input_ids/tokens': tokenizer.convert_ids_to_tokens(dataset['train'][0]['input_ids'][:15]),
    'labels': dataset['train'][0]['labels'][:15],
}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
input_ids/tokens,<s>,ĠMichael,ĠRamirez,Ġ(,ĠIND,IA,Ġ),Ġ,ĊĊ,ĠVIS,UAL,IZ,ATION,Ġ,ĊĊ
labels,-100,1,2,0,0,-100,0,0,-100,0,-100,-100,-100,0,-100


In [235]:
tags = dataset['train'].features['labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None)

In [236]:
tags.names

['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT']

## Create the model for token classification

In [239]:
train_data = dataset['train']
val_data = dataset['test']

In [240]:
len(train_data['input_ids'][0])


1024

In [ ]:
%%capture
!pip install wandb
!pip install accelerate -U
!pip install seqeval

In [241]:
import datasets
from transformers import LongformerForTokenClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report


In [242]:
config = LongformerConfig()
config

LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "onnx_export": false,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [243]:
model = LongformerForTokenClassification.from_pretrained('allenai/longformer-base-4096',
                                                         num_labels=tags.num_classes,
                                                         gradient_checkpointing=False
                                                        )

Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [244]:
model.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "onnx_export": false,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "transformers_version": "4.37.2",
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [245]:
from seqeval.metrics import classification_report

In [246]:
from seqeval.metrics import classification_report
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [ ]:
%%capture
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
from transformers import TrainingArguments

num_epochs = 3
batch_size = 24
logging_steps = len(dataset["train"]) // batch_size

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/models/longformer_baseline/outputs',
    log_level="error",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_steps=logging_steps)

In [247]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [248]:
def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"]) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in data]
    labels = [torch.tensor(item["labels"]) for item in data]


    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)


    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


In [250]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [251]:
%%time
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,F1
0,0.001800,0.001720,0.000000
1,0.000500,0.000722,0.411498
2,0.000400,0.000378,0.836096


CPU times: user 13min 59s, sys: 4min 58s, total: 18min 57s
Wall time: 18min 51s


TrainOutput(global_step=255, training_loss=0.12906800489690082, metrics={'train_runtime': 1131.0296, 'train_samples_per_second': 14.443, 'train_steps_per_second': 0.225, 'total_flos': 1.0656757336264704e+16, 'train_loss': 0.12906800489690082, 'epoch': 3.0})

In [255]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [276]:
def tag_text(text, tags, model, tokenizer):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

In [259]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1362
    })
})

In [265]:
text = 'Michael	Ramirez	(	INDIA	)	\n\n	VISUALIZATION	\n\n	Elements'
text

'Michael\tRamirez\t(\tINDIA\t)\t\n\n\tVISUALIZATION\t\n\n\tElements'

In [266]:
tag_text(text, tags, trainer.model, tokenizer)

Input ids are automatically padded from 25 to 512 to be a multiple of `config.attention_window`: 512


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,ĠMichael,ĉ,Ram,irez,ĉ,(,ĉ,IND,IA,...,VIS,UAL,IZ,ATION,ĉ,ĊĊ,ĉ,E,lements,</s>
Tags,O,O,I-NAME_STUDENT,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O


In [286]:
text = ' '.join(train_data[0]['tokens'][:20])
text

'Design Thinking for innovation reflexion - Avril 2021 - Nathalie Sylla \n\n Challenge & selection \n\n The tool I use'

In [287]:
tag_text(text, tags, trainer.model, tokenizer)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
Tokens,<s>,ĠDesign,ĠThinking,Ġfor,Ġinnovation,Ġreflex,ion,Ġ-,ĠAv,ril,...,ĠChallenge,Ġ&,Ġselection,Ġ,ĊĊ,ĠThe,Ġtool,ĠI,Ġuse,</s>
Tags,O,O,O,O,O,O,O,O,B-NAME_STUDENT,O,...,O,O,O,O,O,O,O,O,O,O


In [290]:
text = ' '.join(train_data[0]['tokens'][:15])
text

'Design Thinking for innovation reflexion - Avril 2021 - Nathalie Sylla \n\n Challenge & selection'

In [291]:
tag_text(text, tags, trainer.model, tokenizer)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
Tokens,<s>,ĠDesign,ĠThinking,Ġfor,Ġinnovation,Ġreflex,ion,Ġ-,ĠAv,ril,...,al,ie,ĠSy,lla,Ġ,ĊĊ,ĠChallenge,Ġ&,Ġselection,</s>
Tags,O,O,O,O,O,O,O,O,O,O,...,B-NAME_STUDENT,O,O,O,O,O,O,O,O,O
